Data from kaggle dataset: https://www.kaggle.com/datasets/arjunbasandrai/25-indian-bird-species-with-226k-images

I used simple CNN with 5 convolutional layers and 3 lenear layers and I achieved 55% accuracy using it (I used free google colab, and I couldn't use more powerfull neural network)

In [ ]:
import pandas as pd
import numpy as np
import time
import gc
from tqdm import tqdm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

%matplotlib inline
sns.set_style('darkgrid')

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sorokinmaksim","key":"9537fa921e351600c84c9f61d1f01441"}'}

In [ ]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 69 Mar 26 07:50 kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d arjunbasandrai/25-indian-bird-species-with-226k-images

100% 4.06G/4.07G [01:18<00:00, 98.7MB/s]
100% 4.07G/4.07G [01:18<00:00, 55.7MB/s]


In [ ]:
! unzip /content/25-indian-bird-species-with-226k-images.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: training_set/Ruddy Shelduck/ML285073431.jpg  
  inflating: training_set/Ruddy Shelduck/ML285073441.jpg  
  inflating: training_set/Ruddy Shelduck/ML285073471.jpg  
  inflating: training_set/Ruddy Shelduck/ML285308851.jpg  
  inflating: training_set/Ruddy Shelduck/ML285974291.jpg  
  inflating: training_set/Ruddy Shelduck/ML286028601.jpg  
  inflating: training_set/Ruddy Shelduck/ML286515491.jpg  
  inflating: training_set/Ruddy Shelduck/ML286585111.jpg  
  inflating: training_set/Ruddy Shelduck/ML288186461.jpg  
  inflating: training_set/Ruddy Shelduck/ML288194981.jpg  
  inflating: training_set/Ruddy Shelduck/ML289170221.jpg  
  inflating: training_set/Ruddy Shelduck/ML291043411.jpg  
  inflating: training_set/Ruddy Shelduck/ML291399111.jpg  
  inflating: training_set/Ruddy Shelduck/ML291400271.jpg  
  inflating: training_set/Ruddy Shelduck/ML291402041.jpg  
  inflating: training_set/Ruddy Shelduck/ML2914

In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.onnx
import torchvision
import helper

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
dir = os.path.abspath(os.curdir)
data_dir=os.path.join(dir, "training_set/")

transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

dataset = datasets.ImageFolder(data_dir, transform=transform)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.66 * len(dataset)), len(dataset) - int(0.66 * len(dataset))])

In [ ]:
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, 2, 1),  # in_channels, out_channels, kernel_size, stride, padding
            nn.MaxPool2d(2),  # kernel_size
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 192, 3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 384, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 14 * 14, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 25),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 256 * 14 * 14)
        x = self.classifier(x)
        return x

In [ ]:
from torch.utils.data import DataLoader, SubsetRandomSampler, ConcatDataset


def setup_dataflow(dataset, train_idx, val_idx):
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)

    train_loader = DataLoader(dataset, batch_size=128, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=256, sampler=val_sampler)

    return train_loader, val_loader

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, f1_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score




learning_rate = 3e-4
train_loader = DataLoader(train_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=256)
net = SimpleConvNet().to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
for epoch in tqdm(range(35)):
    for batch in train_loader:
        # так получаем текущий батч
        X_batch, y_batch = batch
        
        # обнуляем веса
        optimizer.zero_grad()

        # forward + backward + optimize
        y_pred = net(X_batch.to(device))
        loss = loss_fn(y_pred, y_batch.to(device))
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()

100%|██████████| 35/35 [4:37:39<00:00, 475.98s/it]


In [ ]:
pred = []
lables_list = []
with torch.no_grad():
  for data in tqdm(test_loader):
    images, labels = data
    y_pred = net(images.to(device))
    pred.append(F.softmax(y_pred, dim=-1).cpu())
    lables_list.append(labels.cpu())
accuracy_score(torch.argmax(torch.cat(pred, dim=0), 1), torch.cat(lables_list, dim=0))

100%|██████████| 31/31 [03:55<00:00,  7.60s/it]


0.5532440514887531